<a href="https://colab.research.google.com/github/ArashDehghanyan/ml-practicres/blob/main/Save_and_load_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup
##Installs and imports

In [1]:
! pip install pyyaml h5py   # Required to save models in HDF5 format

In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.version.VERSION)

2.8.0


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.
train_labels = train_labels[:1000]

test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0
test_labels = test_labels[:1000]

11501568/11490434 [==============================] - 0s 0us/step


#Define a model
##Start by a simple Sequential model

In [4]:
def create_model():
    # Define model
    model = tf.keras.models.Sequential([
        layers.Dense(512, activation='relu', input_shape=(784,)),
        layers.Dropout(0.2),
        layers.Dense(10)
    ])
    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(0.001),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

# Create a basic model instance
basic_model = create_model()

# Display the model architecture
basic_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#Save checkpoints during training

In [5]:
checkpoint_path = "training_1/checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weight
checkpoint_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True)

# Train model with new callback
basic_model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    # Pass callback to training
    callbacks=[checkpoint_callback]
)

Epoch 1/10
31/32 [============================>.] - ETA: 0s - loss: 1.1605 - sparse_categorical_accuracy: 0.6562
Epoch 1: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 1s 14ms/step - loss: 1.1567 - sparse_categorical_accuracy: 0.6560 - val_loss: 0.7107 - val_sparse_categorical_accuracy: 0.7790
Epoch 2/10
25/32 [======================>.......] - ETA: 0s - loss: 0.4371 - sparse_categorical_accuracy: 0.8838
Epoch 2: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.4157 - sparse_categorical_accuracy: 0.8890 - val_loss: 0.5705 - val_sparse_categorical_accuracy: 0.8200
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.2860 - sparse_categorical_accuracy: 0.9230
Epoch 3: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.2860 - sparse_categorical_accuracy: 0.9230 - val_loss: 0.5290 - val_sparse_categorical_accuracy: 0.8320
Epo

In [6]:
os.listdir(checkpoint_dir)

['checkpoint', 'checkpoint.ckpt.index', 'checkpoint.ckpt.data-00000-of-00001']

##Create an untrained model and evalute it on the test set

In [7]:
untrained_model = create_model()

# Evaluate the model
loss, acc = untrained_model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3753 - sparse_categorical_accuracy: 0.0780 - 178ms/epoch - 6ms/step
Untrained model accuracy:  7.80%


###Load weights from checkpoint and re-evaluate the model

In [8]:
# Load weights
untrained_model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = untrained_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4097 - sparse_categorical_accuracy: 0.8710 - 150ms/epoch - 5ms/step
Restored model accuracy: 87.10%


##Checkpoint options

In [9]:
# Include the epoch in the filename
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback to save model's weight every 5 epochs
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size
)

# Create a new model instance
new_model = create_model()

# Save the weights using (checkpoint_path) format
new_model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with new callback
new_model.fit(
    train_images,
    train_labels,
    epochs=50,
    verbose=1, 
    callbacks=[cp_callback],
    batch_size=batch_size,
    validation_data=(test_images, test_labels)
)

Epoch 1/50
32/32 [==============================] - 2s 23ms/step - loss: 1.2381 - sparse_categorical_accuracy: 0.6380 - val_loss: 0.7532 - val_sparse_categorical_accuracy: 0.7890
Epoch 2/50
32/32 [==============================] - 0s 12ms/step - loss: 0.4382 - sparse_categorical_accuracy: 0.8820 - val_loss: 0.5274 - val_sparse_categorical_accuracy: 0.8410
Epoch 3/50
32/32 [==============================] - 0s 12ms/step - loss: 0.2917 - sparse_categorical_accuracy: 0.9320 - val_loss: 0.4798 - val_sparse_categorical_accuracy: 0.8500
Epoch 4/50
32/32 [==============================] - 0s 13ms/step - loss: 0.2139 - sparse_categorical_accuracy: 0.9480 - val_loss: 0.4523 - val_sparse_categorical_accuracy: 0.8570
Epoch 5/50
26/32 [=======================>......] - ETA: 0s - loss: 0.1625 - sparse_categorical_accuracy: 0.9627
Epoch 5: saving model to training_2/cp-0005.ckpt
32/32 [==============================] - 0s 15ms/step - loss: 0.1662 - sparse_categorical_accuracy: 0.9620 - val_loss: 0.4

In [10]:
os.listdir(checkpoint_dir)

['cp-0000.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0035.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001']

In [11]:
# Select the latest checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
# Create a new model instance
new_untrained_model = create_model()

# Load previously saved weights
new_untrained_model.load_weights(latest)

# Re-evaluate the model
loss, acc = new_untrained_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model accuracy: {:5.2f}%".format(acc * 100))


32/32 - 0s - loss: 0.4839 - sparse_categorical_accuracy: 0.8780 - 189ms/epoch - 6ms/step
Restored model accuracy: 87.80%


#Manually save models

In [13]:
# Save the weights
basic_model.save_weights('./checkpoints/my-checkpoints')

#  Create model
model = create_model()

# Restore model
model.load_weights("./checkpoints/my-checkpoints")

# Evaluate model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4097 - sparse_categorical_accuracy: 0.8710 - 174ms/epoch - 5ms/step
Restored model, accuracy: 87.10%


##SavedModel format

In [14]:
# Create and train a new model instance
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save the rntire model as a savedmodel
! mkdir -p saved_model
model.save("saved_model/my_model")


Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1211 - sparse_categorical_accuracy: 0.6900
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4269 - sparse_categorical_accuracy: 0.8880
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.3040 - sparse_categorical_accuracy: 0.9140
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1975 - sparse_categorical_accuracy: 0.9560
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1474 - sparse_categorical_accuracy: 0.9620
INFO:tensorflow:Assets written to: saved_model/my_model/assets


##Inspect the saved model directory

In [15]:
# contains my_model directory
! ls saved_model

# Contains assets directory, keras_metadata.pb, saved_model.pb and variables folder.
! ls saved_model/my_model/

my_model
assets	keras_metadata.pb  saved_model.pb  variables


##Reload a fresh keras model

In [16]:
new_model = keras.models.load_model("saved_model/my_model")

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))


print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4229 - sparse_categorical_accuracy: 0.8540 - 179ms/epoch - 6ms/step
Restored model, accuracy: 85.40%
(1000, 10)


##HDF5 format

In [18]:
# Create a new model
model = create_model()

# Train model
model.fit(train_images, train_labels, epochs=5, verbose=1)

# Save the whole model to a HDF5 file.
# the .h5 extension indicates that file is in HDF5 format.
model.save("my_model.h5")

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1518 - sparse_categorical_accuracy: 0.6720
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4098 - sparse_categorical_accuracy: 0.8880
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2723 - sparse_categorical_accuracy: 0.9330
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2065 - sparse_categorical_accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1477 - sparse_categorical_accuracy: 0.9670


In [19]:
# Recreate the exact same model, including its weights and optimizer
new_model = keras.models.load_model("my_model.h5")

# Show model structure
new_model.summary()

# Check its accuracy
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model accuracy: {:5.2f}%".format(100 * acc))



Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
32/32 - 0s - loss: 0.4743 - sparse_categorical_accuracy: 0.8460 - 165ms/epoch - 5ms/step
Restored model accuracy: 84.60%


#Saving custom Objects in HDF5 format
##1. Define a custom object
##2. Pass the object to the custom_objects argument when loading the model

In [20]:
# Define a custom layer class
class Linear(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='random_normal',
            trainable=True
        )

        self.b = self.add_weight(
            shape=(self.units,), initializer='random_normal', trainable=True
        )
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(Linear, self).get_config()
        config.update({"units": self.units})
        return config
        
    @classmethod
    def from_config(cls, config):
        return cls(**config)
        

In [21]:
# Create an instance of Linear class, at instantiation we don't know the input shape
linear_layer = Linear(64)

config = linear_layer.get_config()
print(config)
new_layer = linear_layer.from_config(config)

# the layer's weights are created when the first time the layer is called
y = linear_layer(tf.ones((10, 4)))
print(y.numpy().shape)
print(new_layer(tf.ones((5, 2))).numpy().shape)
print("weights:", len(linear_layer.weights))

{'name': 'linear', 'trainable': True, 'dtype': 'float32', 'units': 64}
(10, 64)
(5, 64)
weights: 2


In [54]:
# Create a neural network using custom Linear layer class defined previously
class MLPBlock(keras.Model):
    """Custom multi layer perceptron network."""
    def __init__(self, output_dim, name=None):
        super(MLPBlock, self).__init__(name=name)
        self.output_dim = output_dim
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(self.output_dim)

    def call(self, inputs):
        
        y = self.linear_1(inputs)
        y = tf.nn.relu(y)
        y = self.linear_2(y)
        y = tf.nn.relu(y)
        return self.linear_3(y)
    
    def get_config(self):
        return {"output_dim": self.output_dim, "name": self.name}

# Build an instance of MLP
mlp = MLPBlock(1)
y = mlp(tf.ones((3, 64)))
mlp.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
mlp.fit(tf.ones((3, 64)), tf.ones((3,1)))
print("weights:", len(mlp.weights))
print("Trainable weights:", len(mlp.trainable_weights))

1/1 [==============================] - 0s 498ms/step - loss: 0.9374 - accuracy: 0.0000e+00
weights: 6
Trainable weights: 6


In [55]:
mlp.save("saved_model/mlp", save_format='tf')

AttributeError: ignored

In [36]:
inputs = keras.Input(shape=(32,))
x = Linear(32)(inputs)
x = Linear(64)(x)
x = Linear(32)(x)
outputs = Linear(1)(x)
model = keras.Model(inputs, outputs)
config = model.get_config()
print(config)

custom_objects = {"Linear": Linear}
new_model = keras.Model.from_config(config, custom_objects=custom_objects)

{'name': 'model_4', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 32), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_5'}, 'name': 'input_5', 'inbound_nodes': []}, {'class_name': 'Linear', 'config': {'name': 'linear_29', 'trainable': True, 'dtype': 'float32', 'units': 32}, 'name': 'linear_29', 'inbound_nodes': [[['input_5', 0, 0, {}]]]}, {'class_name': 'Linear', 'config': {'name': 'linear_30', 'trainable': True, 'dtype': 'float32', 'units': 64}, 'name': 'linear_30', 'inbound_nodes': [[['linear_29', 0, 0, {}]]]}, {'class_name': 'Linear', 'config': {'name': 'linear_31', 'trainable': True, 'dtype': 'float32', 'units': 32}, 'name': 'linear_31', 'inbound_nodes': [[['linear_30', 0, 0, {}]]]}, {'class_name': 'Linear', 'config': {'name': 'linear_32', 'trainable': True, 'dtype': 'float32', 'units': 1}, 'name': 'linear_32', 'inbound_nodes': [[['linear_31', 0, 0, {}]]]}], 'input_layers': [['input_5', 0, 0]], 'output_layers': [['linear_32', 0,

In [41]:
model.save_weights("mlp", save_format='tf')

AttributeError: ignored

In [49]:
class SubclassedModel(keras.Model):
    def __init__(self, output_dim, name=None):
        super(SubclassedModel, self).__init__(name=name)
        self.output_dim = output_dim
        self.dense_1 = keras.layers.Dense(64, activation="relu", name="dense_1")
        self.dense_2 = keras.layers.Dense(64, activation="relu", name="dense_2")
        self.dense_3 = keras.layers.Dense(output_dim, name="predictions")

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return x

    def get_config(self):
        return {"output_dim": self.output_dim, "name": self.name}


subclassed_model = SubclassedModel(10)
# Call the subclassed model once to create the weights.
subclassed_model(tf.ones((1, 784)))

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[-0.16164304, -0.07613409, -0.6002283 , -0.21339276, -0.18713108,
         0.0977993 , -0.1912318 ,  0.6535991 , -1.1266735 ,  0.11844908]],
      dtype=float32)>

In [53]:
subclassed_model.save("subclassed_model", save_format='tf')

INFO:tensorflow:Assets written to: subclassed_model/assets
